# Cluster Analysis
<br>
## Task

* Group the facebook posts into groups so that the groups contain posts with similar content.
* This task has no unique solution - the data is not labled so we don't know the correct answer therefore you will not be able to verify how good your model is

## Data
* We prepared two datasets about facebook pages
* The first dataset contains list of pages
* The second dataset contains for each page 100 randomly selected posts
* You will actualy not need to use the pages set unless you come up with some interesting features that will be useful

## Notes
* Create vector reprezentation for the texts
* Split the text to words
* Compute IDF
* Use KMeans algorithm to train a model

## About K-Means
* K-means is a unsupervised learning algorithm that can be used to cluster data into groups
* For details see <a target="_blank" href="https://en.wikipedia.org/wiki/K-means_clustering">wiki</a>

## Documentation
<br>
* Pyspark documentation of DataFrame API is <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html">here</a>

* Pyspark documentation of ML Pipelines library is <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html">here</a>

* Prezentation slides are accessed <a target="_blank" href = "https://docs.google.com/presentation/d/1XNKIfE5Atj_Mzse0wjmbwLecmVs2YkWm9cqOLqDVWPo/edit?usp=sharing">here</a>

### Import functions

In [4]:
from pyspark.sql.functions import col, count, desc, row_number, collect_list, length, array_contains, size
from pyspark.sql.functions import col, count, desc, array_contains, broadcast, explode, length, first, when

from pyspark.sql import Window

from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, Normalizer, SQLTransformer

from pyspark.ml.clustering import KMeans

from pyspark.ml import Pipeline

import random

### Load the data

In [6]:
pages = spark.table('mlprague.facebook_pages')

posts = spark.table('mlprague.facebook_posts')

### You may want to do some exploratory analytics first

hint:
* see how many records you have
* what is the schema of the dataset
* see some records
* use can use printSchema(), show(), count(), or proprietaray function display()

In [8]:
pages.count()

In [9]:
posts.count()

In [10]:
display(pages)

In [11]:
display(posts)

### Extract the features & construct the pipeline

hint
* do vector representation for the texts
 * use: 
 * <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.Tokenizer">Tokenizer</a> 
 * <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.StopWordsRemover">StopWordsRemover</a> 
 * <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.HashingTF">HashingTF</a> to compute term frequency and reduce the space or use the <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.CountVectorizer">CountVectorizer</a>
 * <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.IDF">IDF</a> 
 * <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.Normalizer">Normalizer</a> 
 * <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.clustering.KMeans">KMeans</a> 
* See the slides 83, 84, 85, 101 in the presentation
* after you apply StopWordsRemover it is good to filter out rows with no (or very few) words. You can use the SQLTransformer defined bellow, that filters out all rows that have less than 10 words. This transformer assumes that the output column of StopWordsRemover is named 'noStopWords'. Just add this SQLTransformer to the pipeline right behind the StopwordsRemover

Note
* You may want to play with some input parameters: 
 * number of clusters for KMeans (try 4-8)
 * distanceMeasure for KMeans (default is 'euclidean' but you can try also 'cosine') 
 * numFeatures for HashingTF (try 1000)

In [13]:
# add this to the pipeline to remove empty or short messages

emptyRowsRemover = SQLTransformer(statement='SELECT * FROM __THIS__ where size(noStopWords) >= 10')

In [14]:
tokenizer = Tokenizer(inputCol='message', outputCol='words')

stopWordsRemover = StopWordsRemover(inputCol='words', outputCol='noStopWords')

hashingTF = HashingTF(numFeatures=1000, inputCol='noStopWords', outputCol='hashingTF')

idf = IDF(inputCol='hashingTF', outputCol='idf')

normalizer = Normalizer(inputCol='idf', outputCol='features')

kmeans = KMeans(featuresCol='features', predictionCol='prediction', k=5, seed=1)

pipeline = Pipeline(stages=[tokenizer, stopWordsRemover, emptyRowsRemover, hashingTF, idf, normalizer, kmeans])

model = pipeline.fit(posts)

### Apply the model on the data

hint
* just call transform, since the model is a transformer
* pass the training data as argument to the transform function

In [16]:
predictions = model.transform(posts)

### See how many pages are in your clusters

hint
* you can simply group by the column prediction and count
* the column with the cluster is called prediction by default

In [18]:
display(
  predictions
  .groupBy('prediction')
  .agg(count('*').alias('cnt'))
  .orderBy(desc('cnt'))
)

### See what pages are in your clusters

hint
* just filter the result for specific cluster:
 * filter(col('prediction') == 0) and so on for other clusters

In [20]:
display(
  predictions
  .filter(col('prediction') == 0)
)

In [21]:
display(
  predictions
  .filter(col('prediction') == 1)
)

In [22]:
display(
  predictions
  .filter(col('prediction') == 2)
)

In [23]:
display(
  predictions
  .filter(col('prediction') == 3)
)

In [24]:
display(
  predictions
  .filter(col('prediction') == 4)
)

After playing a little bit with the data and input parameters to the learning algorithms, you might be able to identify that the data gets clustered according to language (with some error of course). By looking at some posts try to identify which clusters belong to english language and save it to a table. You can use this result as input in the next notebook where you will do LDA.

In [26]:
# we generate random string for the table name to avoid collisions
table_name = ''.join([random.choice('abcdefghijklmnoprstuvwxy') for _ in range(20)])

(
  predictions
  .select('page_id', 'message')
  .filter(col('prediction').isin([4])) # here write the number of clusters that belong to english language
  .repartition(32)
  .write
  .mode('overwrite')
  .saveAsTable(table_name)
)

print(table_name)